Natural Language Inferencing (NLI) is a classic NLP (Natural Language Processing) problem that involves taking two sentences (the _premise_ and the _hypothesis_ ), and deciding how they are related- if the premise entails the hypothesis, contradicts it, or neither.

In this tutorial we'll look at the _Contradictory, My Dear Watson_ competition dataset, build a preliminary model using Tensorflow 2, Keras, and BERT, and prepare a submission file.

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
os.environ["WANDB_API_KEY"] = "0" ## to silence warning

In [ ]:
from transformers import AutoTokenizer, TFRobertaModel
import matplotlib.pyplot as plt
import tensorflow as tf

Let's set up our TPU.

In [ ]:
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
except ValueError:
    strategy = tf.distribute.get_strategy() # for CPU and single GPU
    print('Number of replicas:', strategy.num_replicas_in_sync)

## Downloading Data

The training set contains a premise, a hypothesis, a label (0 = entailment, 1 = neutral, 2 = contradiction), and the language of the text. For more information about what these mean and how the data is structured, check out the data page: https://www.kaggle.com/c/contradictory-my-dear-watson/data

In [ ]:
train = pd.read_csv("../input/contradictory-my-dear-watson/train.csv")

We can use the pandas head() function to take a quick look at the training set.

In [ ]:
train.head()

Let's look at one of the pairs of sentences.

In [ ]:
train.premise.values[1]

In [ ]:
train.hypothesis.values[1]

In [ ]:
train.label.values[1]

These statements are contradictory, and the label shows that.

Let's look at the distribution of languages in the training set.

In [ ]:
labels, frequencies = np.unique(train.language.values, return_counts = True)

plt.figure(figsize = (10,10))
plt.pie(frequencies,labels = labels, autopct = '%1.1f%%')
plt.show()

In [ ]:
!pip install nlp
from nlp import load_dataset

In [ ]:
def load_data():
    validation=True
    result = []
    dataset = load_dataset('multi_nli')
    keys = ['train', 'validation_matched','validation_mismatched'] if validation else ['train']
    for k in keys:
        for record in dataset[k]:
            p, h, l = record['premise'], record['hypothesis'], record['label']
            if p and h and l in {0,1,2}:
                result.append((p,h,l,'en'))
    result = pd.DataFrame(result, columns=['premise','hypothesis', 'label','lang_abv'])
    return result

In [ ]:
mnli = load_data()

In [ ]:
mnli = mnli[['premise', 'hypothesis', 'lang_abv', 'label']]
mnli.insert(0, 'language', 'English')
mnli = mnli[['premise', 'hypothesis', 'lang_abv', 'language', 'label']]
mnli.insert(0, 'id', 'xxx')
mnli.head()

In [ ]:
all_training_data = train[['id', 'premise', 'hypothesis','lang_abv', 'language', 'label']]
all_training_data = pd.concat([all_training_data, mnli], axis = 0)
all_training_data

## Preparing Data for Input

To start out, we can use a pretrained model. Here, we'll use a multilingual BERT model from huggingface. For more information about BERT, see: https://github.com/google-research/bert/blob/master/multilingual.md

First, we download the tokenizer.

In [ ]:
model_name = "roberta-large-mnli"
tokenizer = AutoTokenizer.from_pretrained(model_name)

Tokenizers turn sequences of words into arrays of numbers. Let's look at an example:

In [ ]:
def encode_sentence(s):
   return tokenizer(s)['input_ids']

In [ ]:
print(encode_sentence("I love machine learning"))
print(encode_sentence("Machine Learning is great"))

roBERTa uses two kind of input data- input word IDs and input masks.

These allow the model to know that the premise and hypothesis are distinct sentences, and also to ignore any padding from the tokenizer.

Now, we're going to encode all of our premise/hypothesis pairs for input into roBERTa.

In [ ]:
def roberta_encode(hypotheses, premises, tokenizer):
    
  num_examples = len(hypotheses)
  
  sentence1 = tf.ragged.constant([
      encode_sentence(s)
      for s in np.array(hypotheses)])
  sentence2 = tf.ragged.constant([
      encode_sentence(s)
       for s in np.array(premises)])

    
  input_word_ids = tf.concat([sentence1, sentence2], axis=-1)

  input_mask = tf.ones_like(input_word_ids).to_tensor()

  inputs = {
      'input_word_ids': input_word_ids.to_tensor(),
      'input_mask': input_mask,}

  return inputs

In [ ]:
train_input = roberta_encode(all_training_data.premise.values, all_training_data.hypothesis.values, tokenizer)

## Creating & Training Model

Now, we can incorporate the roBERTa transformer into a Keras Functional Model. For more information about the Keras Functional API, see: https://www.tensorflow.org/guide/keras/functional.

This model was inspired by the model in this notebook: https://www.kaggle.com/tanulsingh077/deep-learning-for-nlp-zero-to-transformers-bert#BERT-and-Its-Implementation-on-this-Competition, which is a wonderful introduction to NLP!

In [ ]:
max_len = 250

def max_length(file):
    for key in file.keys():
        file[key] = file[key][:,:max_len]
    return file

def build_model():
    roberta_encoder = TFRobertaModel.from_pretrained(model_name)
    input_word_ids = tf.keras.Input(shape=(max_len,), dtype=tf.int32, name="input_word_ids")
    input_mask = tf.keras.Input(shape=(max_len,), dtype=tf.int32, name="input_mask")
    
    embedding = roberta_encoder([input_word_ids, input_mask])[0]
    output = tf.keras.layers.Dense(3, activation='softmax')(embedding[:,0,:])
    
    model = tf.keras.Model(inputs=[input_word_ids, input_mask], outputs=output)
    model.compile(tf.keras.optimizers.Adam(lr=1e-5), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    
    return model

In [ ]:
with strategy.scope():
    model = build_model()
    model.summary()

In [ ]:
# Freeze model
model.layers[2].trainable=True

In [ ]:
train_input = max_length(train_input)
model.fit(train_input, all_training_data.label.values, epochs = 10, verbose = 1, batch_size = 64, validation_split = 0.2)

In [ ]:
test = pd.read_csv("../input/contradictory-my-dear-watson/test.csv")
test_input = bert_encode(test.premise.values, test.hypothesis.values, tokenizer)

In [ ]:
test.head()

## Generating & Submitting Predictions

In [ ]:
test_input = max_length(test_input)
predictions = [np.argmax(i) for i in model.predict(test_input)]

The submission file will consist of the ID column and a prediction column. We can just copy the ID column from the test file, make it a dataframe, and then add our prediction column.

In [ ]:
submission = test.id.copy().to_frame()
submission['prediction'] = predictions

In [ ]:
submission.head()

In [ ]:
submission.to_csv("submission.csv", index = False)

And now we've created our submission file, which can be submitted to the competition. Good luck!